**Segundo parcial SIS 420**

**Nombre: Yavo Chavez Abigail Zulma**

**Carrera: Ingenieria en diseño y animación digital**

##**4 EN RAYA**

**El aprendizaje** por refuerzo es un método en el que un agente aprende a tomar decisiones para maximizar una recompensa numérica. El agente descubre qué acciones le darán la mayor recompensa a través de la interacción con su entorno. Este método se diferencia del aprendizaje supervisado y no supervisado, ya que el agente no siempre tiene ejemplos del comportamiento deseado y debe aprender de su propia experiencia. El principal desafío es encontrar un equilibrio entre explorar nuevas acciones y explotar las acciones conocidas. El aprendizaje por refuerzo considera todo el problema en su conjunto, en lugar de resolver pequeñas partes por separado.


**El sistema de aprendizaje por refuerzo (AxR) consta de cuatro elementos esenciales:**

1. Política: Define el comportamiento del agente en relación con los estados que percibe y las acciones que puede tomar.

2. Recompensa: Es un valor numérico que el entorno envía al agente, y el objetivo del agente es maximizar esta recompensa.

3. Función de valor: Indica la calidad a largo plazo de un estado, considerando la cantidad total de recompensa que se espera acumular en el futuro empezando en ese estado.

4. Modelo del entorno: Imita el comportamiento del entorno y sirve para planificar acciones considerando estados futuros que todavía no se han experimentado.

Estos elementos son fundamentales para que el agente aprenda a tomar decisiones para maximizar su recompensa a través de la interacción con su entorno.



In [ ]:
import numpy as np

class Board():
    def __init__(self):
        # Inicializa el tablero del juego como una matriz de ceros 4x4
        self.state = np.zeros((4, 4))

    def valid_moves(self):
        # Devuelve una lista de tuplas que representan las posiciones válidas en el tablero donde se puede colocar un símbolo
        return [(i, j) for j in range(4) for i in range(4) if self.state[i, j] == 0]

    def update(self, symbol, row, col):
        # Actualiza el estado del tablero con el símbolo (1 para X, -1 para O) en la fila y columna especificadas
        if self.state[row, col] == 0:
            self.state[row, col] = symbol
        else:
            raise ValueError("¡Movimiento ilegal!")

    def is_game_over(self):
        # Comprueba filas y columnas
        for i in range(4):
            if abs(sum(self.state[i, :])) == 4:
                return np.sign(sum(self.state[i, :]))
            if abs(sum(self.state[:, i])) == 4:
                return np.sign(sum(self.state[:, i]))

        # Comprueba diagonales
        diag1 = sum([self.state[i, i] for i in range(4)])
        diag2 = sum([self.state[i, 3 - i] for i in range(4)])

        if abs(diag1) == 4:
            return np.sign(diag1)
        if abs(diag2) == 4:
            return np.sign(diag2)

         # Comprueba si hay un ganador
        for value in [diag1, diag2] + [self.state.sum(axis=0).sum(), self.state.sum(axis=1).sum()]:
          if abs(value) == 4:
            return np.sign(value)


        # Comprueba si hay empate
        if len(self.valid_moves()) == 0:
            return 0

        # Continuar el juego
        return None

    def reset(self):
        # Resetea el tablero del juego a su estado inicial (todos los ceros)
        self.state = np.zeros((4, 4))

In [ ]:
from tqdm import tqdm

class Game():
    def __init__(self, player1, player2):
        # Asigna los símbolos de los jugadores (1 para player1 y -1 para player2)
        player1.symbol = 1
        player2.symbol = -1
        # Almacena los jugadores y crea un tablero
        self.players = [player1, player2]
        self.board = Board()

    # selfplay: función para que los jugadores jueguen entre sí
    def selfplay(self, rounds=100):
        # Inicializamos una lista para registrar las victorias de cada jugador
        wins = [0, 0]

        # Itera sobre el número de rondas especificadas
        for i in tqdm(range(1, rounds + 1)):
            # Resetea el tablero y los jugadores para una nueva partida
            self.board.reset()

            for player in self.players:
                player.reset()

            game_over = False

            # Bucle principal del juego
            while not game_over:
                for player in self.players:
                    # Cada jugador realiza un movimiento en el tablero
                    action = player.move(self.board)
                    self.board.update(player.symbol, action[0], action[1])

                    # Actualiza el estado del jugador después del movimiento
                    for player in self.players:
                        player.update(self.board)

                    # Verifica si el juego ha terminado
                    if self.board.is_game_over() is not None:
                        game_over = True
                        break

            # Proporciona la recompensa al finalizar la partida
            self.reward()

            # Registra las victorias de cada jugador
            for ix, player in enumerate(self.players):
                if self.board.is_game_over() == player.symbol:
                    wins[ix] += 1
        return wins

    # reward: función para proporcionar recompensas a los jugadores al final de una partida
    def reward(self):
        # Determina el ganador del juego
        winner = self.board.is_game_over()

        if winner == 0: # Si hay empate, ambos jugadores reciben una recompensa de 0.5
            for player in self.players:
                player.reward(0.5)
        else:
            # Si hay un ganador, el jugador que ganó recibe una recompensa de 1, mientras que el otro recibe 0
            for player in self.players:
                if winner == player.symbol:
                    player.reward(1)
                else:
                    player.reward(0)

In [ ]:
class Agent():
    def __init__(self, alpha=0.5, prob_exp=0.55):
        # Inicializa la función de valor como un diccionario vacío
        self.value_function = {}  # tabla con pares estado -> valor
        self.alpha = alpha  # tasa de aprendizaje
        # Inicializa la lista de posiciones para almacenar los estados del juego
        self.positions = []  # guardamos todas las posiciones de la partida
        self.prob_exp = prob_exp  # probabilidad de exploración

    def reset(self):
        # Resetea la lista de posiciones al comienzo de cada partida
        self.positions = []

    # Funcion move: función para que el agente realice un movimiento en el tablero dado
    def move(self, board, explore=True):
        # Obtiene las posibles jugadas válidas en el tablero
        valid_moves = board.valid_moves()

        # EXPLORACIÓN
        if explore and np.random.uniform(0, 1) < self.prob_exp:
            # Se elige una posición aleatoria si se está explorando
            ix = np.random.choice(len(valid_moves))
            return valid_moves[ix]

        # EXPLOTACIÓN
        # Se elige la posición con el valor más alto en la función de valor
        max_value = -1000
        for row, col in valid_moves:
            next_board = board.state.copy()
            next_board[row, col] = self.symbol
            next_state = str(next_board.reshape(4*4))

            value = 0 if self.value_function.get(next_state) is None else self.value_function.get(next_state)

            if value >= max_value:
                max_value = value
                best_row, best_col = row, col
        return best_row, best_col

    def update(self, board):
        # Almacena el estado actual del tablero en la lista de posiciones
        self.positions.append(str(board.state.reshape(4*4)))

    # Obtiene la recompensa del estado actual
    def reward(self, reward):
        # Actualiza los valores de la función de valor al final de la partida
        # Itera sobre los estados de la partida en orden inverso
        for p in reversed(self.positions):
            # Si el estado no está en la función de valor, lo inicializa a 0
            if self.value_function.get(p) is None:
                self.value_function[p] = 0

            # Actualiza el valor del estado utilizando la fórmula de actualización
            self.value_function[p] += self.alpha * (reward - self.value_function[p])
            # Actualiza la recompensa para el próximo estado
            reward = self.value_function[p]

In [ ]:
# Se crea una instancia del agente 1 con una probabilidad de exploración del 55%
Agent1 = Agent(prob_exp=0.55)
# Se crea una instancia del agente 2 con la configuración predeterminada (probabilidad de exploración 0.5)
Agent2 = Agent()

# Se crea una instancia del juego con los dos agentes creados anteriormente
game = Game(Agent1, Agent2)

# Los agentes juegan entre sí un número especificado de rondas (400 en este caso)
game.selfplay(400)


100%|██████████| 400/400 [00:11<00:00, 36.06it/s]


[137, 102]

In [ ]:
import pandas as pd

# Ordenamos la función de valor del Agente1 en orden descendente según los valores
funcion_de_valor = sorted(Agent1.value_function.items(), key=lambda kv: kv[1], reverse=True)

# Creamos un DataFrame de pandas a partir de la función de valor ordenada
# con dos columnas: 'Estado' que contiene los estados del juego y 'Valor' que contiene los valores asociados
tabla = pd.DataFrame({'Estado': [x[0] for x in funcion_de_valor], 'Valor': [x[1] for x in funcion_de_valor]})

# Mostramos la tabla
print(tabla)

                                                 Estado  Valor
0     [-1.  0.  0.  1.  1.  1.  1.  1.  0. -1.  1. -...   0.75
1     [ 1. -1. -1.  1.  0. -1. -1. -1.  1.  1.  1.  ...   0.50
2     [ 0.  1. -1.  1.  0. -1.  1. -1.  0.  1. -1.  ...   0.50
3     [-1.  0.  1. -1. -1.  0.  0.  1.  1.  1.  1.  ...   0.50
4     [ 0.  0.  0.  0.  1.  1.  1.  1.  1.  0. -1. -...   0.50
...                                                 ...    ...
4759  [ 0. -1. -1. -1. -1. -1.  1.  1.  0.  0.  1.  ...   0.00
4760  [ 0. -1. -1. -1.  0. -1.  1.  1.  0.  0.  1.  ...   0.00
4761  [ 0. -1. -1. -1.  0. -1.  1.  1.  0.  0.  1.  ...   0.00
4762  [ 0.  0. -1. -1.  0. -1.  1.  1.  0.  0.  1.  ...   0.00
4763  [ 0.  0. -1. -1.  0. -1.  1.  1.  0.  0.  0.  ...   0.00

[4764 rows x 2 columns]


In [ ]:
import pickle #Se utiliza para serializar y deserializar objetos en Python.

with open('agente.pickle', 'wb') as handle: #abre un archivo llamado en modo escritura binaria.
    pickle.dump(Agent1.value_function, handle, protocol=pickle.HIGHEST_PROTOCOL) # guarda el objeto Agent1 en el archivo abierto utilizando la función pickle.dump